In [61]:
!pip install -U langchain sentence-transformers faiss-cpu


In [62]:
import openai
import sys
import os

In [75]:
#os.environ["OPENAI_API_KEY"] =


In [64]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/Hands-On Large Language  Models.pdf')

pages = loader.load()
#print(len(pages))
#print(type(pages))
#print(pages[10])

page = pages[10]

#print(page.page_content[:500])

#print(page.metadata)


In [65]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(pages)
print(f"num of docs: {len(docs)}")
print(f"num of tokens: {len(docs[0].page_content.split())}")



num of docs: 478
num of tokens: 147


In [66]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

#embedding = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embedding)


***Langchain Chatbot***

In [72]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
#from langchain.llms import HuggingFaceHub
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

#pipe = pipeline("text2text-generation", model="google/flan-t5-base")

pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=512,
    truncation=True
)

llm = HuggingFacePipeline(pipeline=pipe)


retriever = db.as_retriever()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#llm = HuggingFaceHub(
 #   repo_id="google/flan-t5-base",
  #  huggingfacehub_api_token="",
   # task="text2text-generation",
    #model_kwargs={"temperature": 0.0}
#)

qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)


Device set to use cpu


In [74]:
query = "What is Language AI?"
response = qa.run(query)
print(response)


The term artificial intelligence (AI) is often used to describe computer systems dedicated to performing tasks close to human intelligence, such as speech recognition, language translation, and visual perception. The term Language AI refers to a subfield of AI that focuses on developing technologies capable of understanding, processing, and generating human language. The term Language AI can often be used interchangeably with natural language processing (NLP) with the continued success of machine learning methods in tackling language processing problems. We use the term Language AI to encompass technologies that technically might not be LLMs but still have a significant impact on the field, like how retrieval systems can give LLMs superpowers (see Chapter 8).
